## Interactive Map of Washington State River Channel Change

import packages

In [1]:
import pyproj
import json
import bokeh
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_file, show
from bokeh.tile_providers import CARTODBPOSITRON, get_provider
from bokeh.models import GeoJSONDataSource, Patches, ColumnDataSource, HoverTool
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon

Convert seattle lat long into web mercator using pyproj

In [2]:
web_merc = pyproj.Proj(projparams = 'epsg:3857')
InputGrid = pyproj.Proj(projparams = 'epsg:4326')
lat, lon = 47.6062, -122.3321
# x1, y1 = -11705274.6374,4826473.6922
wb_seattle_x, wb_seattle_y = pyproj.transform(InputGrid, web_merc, lat, lon)

<ipython-input-2-51617471e3b2>:5: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  wb_seattle_x, wb_seattle_y = pyproj.transform(InputGrid, web_merc, lat, lon)


Import data

In [3]:
results_fn = r'study_data\gauge_results.csv'
results_df = pd.read_csv(results_fn)
results_df['geometry'] = [Point(lon, lat) for lon, lat in zip(results_df.lon, results_df.lat)]
results_web_merc = gpd.GeoDataFrame(
    results_df,
    crs='EPSG:4326',
    geometry='geometry'
    ).to_crs('epsg:3857')

In [4]:
water_fn = r'map_data\WA_hydrography\WA_hydrography_water_bodies.shp'
water_gdf = gpd.read_file(water_fn)
water_gdf_webmerc = water_gdf.to_crs('epsg:3857')
rivers_gdf = water_gdf_webmerc[water_gdf_webmerc.WB_CART__1 == 'Stream/river']

create psuedo dot size to render visible channel change

In [5]:
results_web_merc['dotplot_norm_range_CE'] = results_web_merc.norm_range_CE*25

create a tooltip column

In [6]:
# results_web_merc.assign(fonts = lambda x: '<i>%'

export as geojson (commnet out if already exists)

In [7]:
# geojson_fn = r'study_data\gage_results.geojson'
# results_web_merc.to_file(geojson_fn, driver = "GeoJSON") #comment out if already saved

load geojson file and dump using bokeh

In [8]:
geojson_fn = r'study_data\gage_results.geojson'
with open(geojson_fn) as geofile:
    j_file_data = json.load(geofile)
    
geo_source_results = GeoJSONDataSource(geojson=json.dumps(j_file_data))

plot rivers (https://automating-gis-processes.github.io/2017/lessons/L5/interactive-map-bokeh.html)

In [9]:
#convert polygon data into something bokeh can understand
def getPolyCoords(row, geom, coord_type):
    """Returns the coordinates ('x' or 'y') of edges of a Polygon exterior"""

    # Parse the exterior of the coordinate
    exterior = row[geom].exterior

    if coord_type == 'x':
        # Get the x coordinates of the exterior
        return list( exterior.coords.xy[0] )
    elif coord_type == 'y':
        # Get the y coordinates of the exterior
        return list( exterior.coords.xy[1] )
    
# Get the Polygon x and y coordinates
rivers_gdf['x'] = rivers_gdf.apply(getPolyCoords, geom='geometry', coord_type='x', axis=1)
rivers_gdf['y'] = rivers_gdf.apply(getPolyCoords, geom='geometry', coord_type='y', axis=1)

C:\Users\sahrendt\Anaconda3\envs\WA-channel-change-plots\lib\site-packages\geopandas\geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [10]:
# Make a copy, drop the geometry column and create ColumnDataSource
rg_df = rivers_gdf.drop('geometry', axis=1).copy()
rgsource = ColumnDataSource(rg_df)

In [11]:
results_web_merc.columns

Index(['#', 'USGS Gagename', 'USGS Gageno', 'lat', 'lon', 'max_cap_flood_flow',
       'range_FE', 'norm_range_FE', 'range_CE', 'norm_range_CE',
       'return_period', 'FS_m', 'category', 'channel_fig_url', 'geometry',
       'dotplot_norm_range_CE'],
      dtype='object')

In [12]:
TOOLTIPS = """
    <div>
        <div>
            <img
                src="@channel_fig_url" height="200" alt="@imgs" width="500"
                style="float: left; margin: 0px 0px 0px 0px;"
                border="2"
            ></img>
        </div>
    </div>
"""

plot map with gauging sites

In [13]:
output_file(r"docs\channel_change_map.html")

tile_provider = get_provider(CARTODBPOSITRON)
seattle_utm = (449799.79, 5272748.59)
# range bounds supplied in web mercator coordinates
p = figure(x_range=(wb_seattle_x - 500000, wb_seattle_x + 500000), y_range=(wb_seattle_y - 450000, wb_seattle_y + 300000),
           x_axis_type="mercator", y_axis_type="mercator",
#            tooltips=TOOLTIPS,
           title='western washington channel change',
#            plot_width=800, plot_height=800, 
           active_scroll = "wheel_zoom",
           min_border_left = 100,
           min_border_right = 100,
          )
hover_tool = HoverTool(tooltips=TOOLTIPS,
                       attachment='right')
p.tools.append(hover_tool)
#set hovertool with 'attachment='left'' feature smehow
# p.circle(x='x', y='y', size=15, color='tomato', alpha=0.7, source=geo_source_results)
p.circle(x='x', y='y', size='dotplot_norm_range_CE', color='tomato', alpha=0.7, source=geo_source_results)
# p.circle(x='x', y='y', size=15, color='norm_range_CE', alpha=0.7, source=geo_source_results)
# p.patches('x', 'y', source=rgsource, fill_color='blue', fill_alpha=1.0, line_color="blue", line_width=0.03)
p.add_tile(tile_provider)

show(p)

In [20]:
hover_tool = HoverTool(tooltips=TOOLTIPS,
                       attachment='right')
p.tools.append(hover_tool)